# Load Dependencies

In [1]:
from dotenv import load_dotenv

load_dotenv()
from transformers import AutoModelForObjectDetection, AutoImageProcessor, pipeline

import os
from tqdm import tqdm
import torch.nn as nn
import torch
from torchvision.transforms import v2
import torchvision.models as models
from pathlib import Path

from PIL import Image, ImageDraw, ImageFont
import json
import numpy as np
from transformers import pipeline

# import custom file

from source_code.processor import vid_img_converter as v2img

In [2]:
def crop_objects_from_image(image: Image.Image, labels: list, padding: int = 10):
    """
    Crops objects from an image using bounding boxes with optional padding.
    
    Args:
        image (PIL.Image): The source image.
        labels (list): List of dicts with 'box' key containing 'xmin', 'ymin', 'xmax', 'ymax'.
        padding (int): Number of pixels to pad around the bounding box.
        
    Returns:
        List[PIL.Image]: Cropped image patches.
    """
    width, height = image.size
    cropped_images = []

    for obj in labels:
            box = obj['box']
            xmin = max(box['xmin'] - padding, 0)
            ymin = max(box['ymin'] - padding, 0)
            xmax = min(box['xmax'] + padding, width)
            ymax = min(box['ymax'] + padding, height)

            cropped = image.crop((xmin, ymin, xmax, ymax))
            cropped_images.append(cropped)

    return cropped_images

def load_model(checkpoint_path, num_classes, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # LOAD THE CLASSIFIER MODEL
    model = models.mobilenet_v3_large()
    model.classifier[3] = nn.Linear(in_features=1280, out_features=num_classes)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    return model

font = ImageFont.truetype("./ARIAL.TTF",size=20)
def plot_results(image, results, threshold=0.6):
    image = Image.fromarray(np.uint8(image))
    draw = ImageDraw.Draw(image)
    width, height = image.size

    for result in results:
        score = result["score"]
        label = result["label"]
        box = list(result["box"].values())
        if label in ['motorcycle','autorickshaw','rider']:
            outline_color = 'red'
        else:
            outline_color = 'yellow'
        if score > threshold:
            x1, y1, x2, y2 = tuple(box)
            draw.rectangle((x1, y1, x2, y2), outline=outline_color, width=3)
            draw.text((x1 + 5, y1 - 20), label, fill="white",font=font)
            draw.text((x1 + 5, y1 + 10), f"{score:.2f}", fill="green" if score > 0.7 else "red")

    return image

def infer_image(model, image: Image.Image, device='cuda' if torch.cuda.is_available() else 'cpu'):
    transform_x = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    class_names={
        0: 'Coupe',
        1: 'Sedan',
        2: 'Cab',
        3: 'Convertible',
        4: 'SUV',
        5: 'Minivan',
        6: 'Hatchback',
        7: 'Other',
        8: 'Van',
        9: 'Wagon'
    }
    input_tensor = transform_x(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        output = model(input_tensor)
        predicted_class = output.argmax(dim=1).item()

    return class_names[predicted_class]

# 1. Video -> Image frames
- set the `VIDEO_GDRIVE_ID` on the `.env` file
-  download the video using `gdown` and save if to `data/test_video.mp4`
- convert video to images in folder `"data/frame_dir"`

In [ ]:
vid_id = os.environ.get("VIDEO_GDRIVE_ID")
vid_path = 'data/test_video.mp4'
v2img.gdown.download(id=vid_id, output=vid_path, quiet=False)

In [ ]:
vid2img.video_to_images(vid_path,"data/frame_dir/")

# 2. Object Detection on Image Frames

In [7]:
image_folder = "./data/frame_dir"
json_output_folder = "./output/frame_json"
batch_size = 8  # adjust as needed

In [8]:
# 🧪 Create output folder
os.makedirs(json_output_folder, exist_ok=True)

# 📦 Load pipeline

# Load processor and model manually
processor = AutoImageProcessor.from_pretrained("izzako/detr-resnet-50-finetuned-IDD_Detection")
model = AutoModelForObjectDetection.from_pretrained(
    "izzako/detr-resnet-50-finetuned-IDD_Detection",
    torch_dtype="auto",          # Let HF decide best dtype
    device_map="auto"            # Avoid meta tensor issues
)

# Build the pipeline
obj_detector = pipeline(
    "object-detection",
    model=model,
    image_processor=processor,
    # device=0  # or -1 for CPU
)


Device set to use cuda:0


In [9]:
# 📂 Load image paths
image_paths = sorted(list(Path(image_folder).glob("*.jpg")))


# 🔁 Batch inference and save to JSON
for i in tqdm(range(0, len(image_paths), batch_size),):
    
    batch_paths = image_paths[i:i+batch_size]
    batch = [Image.open(p).convert("RGB") for p in batch_paths]
    results = obj_detector(batch)

    for path, prediction in zip(batch_paths, results):
        filename = Path(path).stem + ".json"
        output_path = Path(json_output_folder) / filename

        # 📝 Save prediction to JSON
        with open(output_path, "w") as f:
            json.dump(prediction, f, indent=2)

        # print(f"✅ Saved: {output_path}")

  0%|          | 0/740 [00:00<?, ?it/s]

100%|██████████| 740/740 [05:46<00:00,  2.13it/s]


# 3. Car Classifier on Detected Bounding Box

In [5]:
image_folder = "./data/frame_dir"
detection_folder = "./output/frame_json"
batch_size = 8  # adjust as needed

model = load_model("./output/mobilenet_v3_large_checkpoint_10.pth", 10)

labeled_folder =  "./output/frame_label"
os.makedirs(labeled_folder,exist_ok=True)


label_folder = "./output/frame_label"
rec_plotted_img =  "./output/pred_image"

smoothed_dir = './output/smoothed_frame_label'
os.makedirs(smoothed_dir,exist_ok=True)

In [12]:
# RENAME DETECTED IMAGE TO CAR TYPE USING CLASSIFIER

for j,label in enumerate(tqdm(sorted(os.listdir(detection_folder)))):
    results = json.load(open(os.path.join(detection_folder,label),'r'))
    image = Image.open(os.path.join(image_folder,sorted(os.listdir(image_folder))[j]))
    cropped_imgs = crop_objects_from_image(image=image, labels=results, padding=20)
    for i,result in enumerate(results):
        if result['label']!='car':continue
        pred_label = infer_image(model, cropped_imgs[i])
        result['label']=pred_label

    with open(os.path.join(labeled_folder,label), "w") as f:
        json.dump(results, f, indent=2)

  0%|          | 0/5920 [00:00<?, ?it/s]/projectnb/llamagrp/izzan/env/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


100%|██████████| 5920/5920 [02:48<00:00, 35.16it/s]


## smoothing the bounding boxes

In [21]:
def iou(box1, box2):
    """Compute IoU between two boxes in [xmin, ymin, xmax, ymax] format."""
    xA = max(box1['xmin'], box2['xmin'])
    yA = max(box1['ymin'], box2['ymin'])
    xB = min(box1['xmax'], box2['xmax'])
    yB = min(box1['ymax'], box2['ymax'])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1['xmax'] - box1['xmin']) * (box1['ymax'] - box1['ymin'])
    box2Area = (box2['xmax'] - box2['xmin']) * (box2['ymax'] - box2['ymin'])

    return interArea / float(box1Area + box2Area - interArea + 1e-6)

def smooth_boxes_across_frames(frames, iou_thresh=0.5, smoothing_weight=0.6):
    """
    Match detections across frames using IoU and apply smoothing.
    :param frames: List of detection results per frame.
    :return: List of smoothed detection results.
    """
    smoothed_frames = [frames[0]]  # first frame remains unchanged

    for t in range(1, len(frames)):
        prev_detections = smoothed_frames[-1]
        curr_detections = frames[t]
        matched = set()
        new_detections = []

        for curr in curr_detections:
            best_iou = 0
            best_prev = None

            for prev in prev_detections:
                # if prev['label'] != curr['label']:
                #     continue
                iou_score = iou(prev['box'], curr['box'])
                if iou_score > best_iou:
                    best_iou = iou_score
                    best_prev = prev

            if best_iou > iou_thresh and best_prev:
                # Apply smoothing on matched boxes
                smoothed_box = {}
                for key in ['xmin', 'ymin', 'xmax', 'ymax']:
                    smoothed_box[key] = (
                        smoothing_weight * curr['box'][key] +
                        (1 - smoothing_weight) * best_prev['box'][key]
                    )
                new_detections.append({'label': curr['label'], 'box': smoothed_box,'score':curr['score']})
                matched.add(id(best_prev))
            else:
                # Unmatched, keep as-is
                new_detections.append(curr)

        smoothed_frames.append(new_detections)

    return smoothed_frames

In [24]:
# SMOOTHING
results = []
for i in tqdm(range(len(os.listdir(label_folder)))):
    results.append(json.load(open(os.path.join(label_folder,sorted(os.listdir(label_folder))[i]),'r')))

smoothed_results = smooth_boxes_across_frames(results, iou_thresh=0.8, smoothing_weight=0.6)
for i,filename in enumerate(tqdm(sorted(os.listdir(label_folder)))):
    json.dump(smoothed_results[i],open(os.path.join(smoothed_dir,filename),'w'),indent=2)

100%|██████████| 5920/5920 [00:16<00:00, 366.36it/s]


## plot to images

In [6]:
# PLOT THE CLASSIFIER RESULT TO IMAGES WITH BOUNDING BOX
os.makedirs(rec_plotted_img,exist_ok=True)
for i in tqdm(range(len(os.listdir(smoothed_dir)))):
    image_name=sorted(os.listdir(image_folder))[i]
    image = Image.open(os.path.join(image_folder,image_name))
    results = json.load(open(os.path.join(smoothed_dir,sorted(os.listdir(smoothed_dir))[i]),'r'))
    plot_results(image, results, threshold=0.5).save(os.path.join(rec_plotted_img,image_name))

100%|██████████| 5920/5920 [02:53<00:00, 34.17it/s]


# 4. Predicted Frames to Video

In [7]:
v2img.images_to_video('output/pred_image', 'output/smooth_output_video.mp4')

Creating video of 197.00 seconds at 30.05 FPS.


  0%|          | 24/5920 [00:00<00:50, 117.46it/s]

100%|██████████| 5920/5920 [00:53<00:00, 110.31it/s]

Video saved to output/smooth_output_video.mp4
